# Quantum Error Correction for Dummies (QEC4D)
# Chapter I - The 3-qbit bit flip repetition code

To kick-start our series on quantum error correction (QEC) let's look at the simplest error-correction code there is: the repetition code protecting against bit-flip (X) errors.
## 1) Overview
The quantum repetition code (QRC) ensures information integrity by encoding one ***logical*** qbit of information into 3 ***physical*** qbits.
To do so, it relies on two assumptions:
- Only a single error can occur in each code word.
- The error type can only be an ***X-error aka a bit-flip*** (no Y-errors aka phase flips).

So what are our goals with QRC?
1. Identify whether an error has occurred (or not, we could get lucky).
2. If an error occurred, identify which physical qbit was affected (this is done by studying the ***error syndrome***, more on this later).
3. If an error occurred, correct it.

In [1]:
# First, a few imports
from copy import deepcopy
from qiskit import *
from qiskit import Aer
from qiskit.visualization import plot_histogram
from typing import Tuple

## 2) Building our circuit

### 2-A) Encoding
The first step is to build an encoding. This process turns our logical qbit into 3 physical qbits using repetition. The very first qbit at index $0$ is the one which contains the logical information we're interested in. We apply CNOT conditioned on it to both qbits at index 1 and 2 such that after this encoding all three qbits are necessarily in the same state.

In [2]:
def encode_1_into_3(circuit:QuantumCircuit) -> QuantumCircuit:
    """Encode 1 logical qbit (at index 0) into 3 physical qubits."""
    circuit.cnot(0,1)
    circuit.cnot(0,2)
    return circuit

And we visualise the resulting circuit.

In [3]:
c = QuantumCircuit(QuantumRegister(3))
encoding_circuit = encode_1_into_3(c)
encoding_circuit.draw('text')

q0_0: ──■────■──
      ┌─┴─┐  │  
q0_1: ┤ X ├──┼──
      └───┘┌─┴─┐
q0_2: ─────┤ X ├
           └───┘

Next we use this encoding to build a QRC circuit. We need to add a few things such as parity checks to allow for ulterior processing.

In [4]:
def repetition_code_3(logical_qb_val:int=0, with_error:bool=False, qb_idx:int=-1) -> QuantumCircuit:
    """Build a repetition code circuit.
     The circuit is made of 3 physical qubits encoding the single logical qubit, two ancilla 
     quantum qbits for parity check measurements and two classical bits to receive the results form 
     the parity checks."""
    
    # Encode the logical qubit information into three physical qubits
    qr1 = QuantumRegister(3)
    circuit = QuantumCircuit(qr1)

    if logical_qb_val == 1:
        circuit.x(0)
        circuit.barrier(qr1) #for visualisation only

    circuit = encode_1_into_3(circuit)
    circuit.barrier(qr1) #for visualisation only

    # If there is an error, introduce it here on deisred qbit
    if with_error:
        circuit.x(qb_idx)
        circuit.barrier(qr1) #for visualisation only

    # Add another quantum (ancilla) register for parity checks
    qr2 = QuantumRegister(2)
    circuit.add_register(qr2)
    circuit = parity_checks_qba_qbb(circuit,0,1,3) # Perform parity check on the qubit pair 0-1
    circuit = parity_checks_qba_qbb(circuit,0,2,4) # Perform parity check on the qubit pair 1-2
    circuit.barrier(qr1,qr2) #for visualisation only

    return circuit

def parity_checks_qba_qbb(circuit:QuantumCircuit, qba_idx:int, qbb_idx:int, qb_out:int) -> QuantumCircuit:
    """Perform parity check on the qubit pair a-b."""
    circuit.cnot(qba_idx,qb_out)
    circuit.cnot(qbb_idx,qb_out)
    return circuit

We can now print the circuit we have built to encode this single $0$ logical qbit.

In [5]:
base_circuit = repetition_code_3()
base_circuit.draw('text')

░                      ░ 
q1_0: ──■────■───░───■─────────■────────░─
      ┌─┴─┐  │   ░   │         │        ░ 
q1_1: ┤ X ├──┼───░───┼────■────┼────────░─
      └───┘┌─┴─┐ ░   │    │    │        ░ 
q1_2: ─────┤ X ├─░───┼────┼────┼────■───░─
           └───┘ ░ ┌─┴─┐┌─┴─┐  │    │   ░ 
q2_0: ─────────────┤ X ├┤ X ├──┼────┼───░─
                   └───┘└───┘┌─┴─┐┌─┴─┐ ░ 
q2_1: ───────────────────────┤ X ├┤ X ├─░─
                             └───┘└───┘ ░

A bit more involved but still straightforward, no? We've just added two layers of parity checks which basically ensure that all information qbits are of the same value, if they aren't then this will be picked up by the parity checks when they are measured.

### 2-B) Measuring

In [6]:
def measure_repetition_code_3(circuit:QuantumCircuit, nb_shots:int=1024) -> Tuple[QuantumCircuit, dict]:
    """Performs the measurements on both ancilla qubits.
     Returns the results obtained together with their number of occurences."""
    
    c = deepcopy(circuit)
    
    # Add a classical register to receive the measurement results from ancilla qubits
    cr = ClassicalRegister(2)
    c.add_register(cr)
    
    c.measure(3,0) #Measures ancilla qbit 3, puts the result into cbit 0
    c.measure(4,1)  #Measures ancilla qbit 4, puts the result into cbit 2

    counts = simulate_measurements(c, nb_shots)

    return c, counts

def simulate_measurements(circuit:QuantumCircuit, nb_shots:int=1024) -> dict:
    """Simulates measurement results."""
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = backend_sim.run(transpile(circuit, backend_sim), shots=nb_shots)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(circuit)
    return counts

Now let's measure both ancilla qubits to get the result. Here we should obtain 00 as all qubits were initialised at $0$ and no error was introduced.

In [7]:
error_free_circuit, counts = measure_repetition_code_3(base_circuit)

print(f"Measurement outputs and their number of occurences: {counts}")
error_free_circuit.draw('text')

Measurement outputs and their number of occurences: {'00': 1024}


░                      ░       
q1_0: ──■────■───░───■─────────■────────░───────
      ┌─┴─┐  │   ░   │         │        ░       
q1_1: ┤ X ├──┼───░───┼────■────┼────────░───────
      └───┘┌─┴─┐ ░   │    │    │        ░       
q1_2: ─────┤ X ├─░───┼────┼────┼────■───░───────
           └───┘ ░ ┌─┴─┐┌─┴─┐  │    │   ░ ┌─┐   
q2_0: ─────────────┤ X ├┤ X ├──┼────┼───░─┤M├───
                   └───┘└───┘┌─┴─┐┌─┴─┐ ░ └╥┘┌─┐
q2_1: ───────────────────────┤ X ├┤ X ├─░──╫─┤M├
                             └───┘└───┘ ░  ║ └╥┘
c0: 2/═════════════════════════════════════╩══╩═
                                           0  1

We now have in `counts` all the measurement outcomes together with the number of times they happened. In this case, the circuit being deterministic, one outcome happened 100% of the time while all other possible outcomes happened 0% of the time. However note that this is not the case if the circuit is probabilistic. In that case, you would have different outcomes and different number of occurrences for each. Based on that scenario, we just need one extra step to make our process general enough, we need to look at the outcome with highest probability.

In [8]:
def most_common_output(counts:dict) -> str:
    """ Provides the (q/c)-bit 0>n encoding of the most measured output."""
    return max(counts, key=counts.get)[::-1]

In [9]:
most_frequent_out = most_common_output(counts)
print(f"The measurement outcome obtained most frequently is: {most_frequent_out}")

The measurement outcome obtained most frequently is: 00


That's it, we're now done with the very first step of the process which is building the circuit and running it error-free to see the behaviour that we expect. And we've confirmed that if the circuit has no errors, an input of $\ket{000}$ will result in a measurement outcome of $00$.

## 3) Introducing and identifying errors

At this stage, we want to look at the outputs from our measurements for different errors. These outputs which correspond to measuring the parity-check results from the ancilla qbits are called the ***error syndrome***.

Looking closely at the circuit, we see that the web of CNOTs is what causes a chain reaction in the case of an error occuring. With this in mind, let's construct our single bit-flip truth table (i.e. we're only allowed to have one non-zero value in the input string of bits).

Note that we here describe the bitstrings and measurement outcomes assuming qbit/cbit $0$ is on the left and qbit/cbit $n$ is on the right. Qiskit displays the results of measurements in the reverse order so we need to reverse the string of measurement outcomes later on in the process to ensure consistence.

| Input bitstring | Error syndrome (parity-check measurement outcome) |
|-----------------|---------------------|
| 000             | 00 (no error)       |
| 100             | 11                  |
| 010             | 10                  |
| 001             | 01                  |

This is the theory. Let's now compare it to what happens when we run the circuit on the simulator.

In [10]:
error_indexes = [0,1,2]

example_circ = None
for e_idx in error_indexes:
    circuit = repetition_code_3(with_error=True, qb_idx=e_idx)
    circuit_measured, counts = measure_repetition_code_3(circuit)
    most_frequent_out = most_common_output(counts)
    print(f"For a bitflip error on qbit index {e_idx}, the most frequent syndrome was: {most_frequent_out}")
    example_circ = circuit

example_circ.draw('text')

For a bitflip error on qbit index 0, the most frequent syndrome was: 11
For a bitflip error on qbit index 1, the most frequent syndrome was: 10
For a bitflip error on qbit index 2, the most frequent syndrome was: 01


░       ░                      ░ 
q21_0: ──■────■───░───────░───■─────────■────────░─
       ┌─┴─┐  │   ░       ░   │         │        ░ 
q21_1: ┤ X ├──┼───░───────░───┼────■────┼────────░─
       └───┘┌─┴─┐ ░ ┌───┐ ░   │    │    │        ░ 
q21_2: ─────┤ X ├─░─┤ X ├─░───┼────┼────┼────■───░─
            └───┘ ░ └───┘ ░ ┌─┴─┐┌─┴─┐  │    │   ░ 
q22_0: ─────────────────────┤ X ├┤ X ├──┼────┼───░─
                            └───┘└───┘┌─┴─┐┌─┴─┐ ░ 
q22_1: ───────────────────────────────┤ X ├┤ X ├─░─
                                      └───┘└───┘ ░

If we recap, so far we can:
- Encode the information of a single logical qbit into a triplet of physical qbits using the repetition code.
- Detect if a single bit-flip error has occured.
- Based on the syndrome (aka measurement outcome), identify which physical qbit has suffered from a bit-flip error.

What's left to do?

Well, now that we can tell if an error happened (or not) and if that error happened, which error it was, the only thing left to do is to correct the error that we've spotted and get back an error-free code-word.

In [11]:
def which_qbit_suffered_an_error(syndrome:str) -> Tuple[bool, int]:
    """Based on the syndrome, determines which qbit was affected by a bit-flip error."""
    error_happened = True
    first_parity_check_val = int(syndrome[0])
    second_parity_check_val = int(syndrome[1])
    if first_parity_check_val == 1: # can be either 11 or 10
        if second_parity_check_val == 1: # 11 case
            error_index = 0 # error occurred on qubit 0, i.e. the input string was 100 instead of 000
        else: # 10 case
            error_index = 1 # error occurred on qubit 1, i.e. the input string was 010 instead of 000
    # here can only be 00 or 01
    elif second_parity_check_val == 1: # we try to figure out if it's 01
        error_index = 2 # error occurred on qubit 2, i.e. the input string was 001 instead of 000
    else: # only possibility left is 00 = no error
        error_index = -1 # just our convention, could be anything, we're not going to use it
        error_happened = False

    return error_happened, error_index

## 4) Correcting errors
The last step of this process is to apply an error correction action based on the syndrome that was detected.

In [12]:
def correct_error(circuit:QuantumCircuit, syndrome:str) -> QuantumCircuit:
    c = deepcopy(circuit)
    error_happened, error_index = which_qbit_suffered_an_error(syndrome)
    if error_happened:
        c.x(error_index)
    return c

We update the measurement process to now look at the three qbits encoding the information rather than the parity check ones.

In [13]:
def measure_corrected_circuit(circuit:QuantumCircuit, nb_shots:int=1024) -> Tuple[QuantumCircuit, dict]:
    """Performs the measurements on both ancilla qubits.
     Returns the results obtained together with their number of occurences."""
    
    c = deepcopy(circuit)
    
    # Add a classical register to receive the measurement results from ancilla qubits
    cr = ClassicalRegister(3)
    c.add_register(cr)
    
    c.measure(0,0)
    c.measure(1,1)
    c.measure(2,2)

    counts = simulate_measurements(c, nb_shots)

    return c, counts

And now let's apply our scheme, namely identifying the error based on the syndrome and correcting it.

In [14]:
logical_values = [0,1]
error_indexes = [0,1,2]

example_circ = None
for log_val in logical_values:
    for e_idx in error_indexes:
        circuit = repetition_code_3(logical_qb_val=log_val, with_error=True, qb_idx=e_idx)
        circuit_measured, counts = measure_repetition_code_3(circuit)
        most_frequent_out = most_common_output(counts)
        syndrome = most_common_output(counts)
        corrected_circuit = correct_error(circuit, syndrome)
        corrected_circuit, corrected_counts = measure_corrected_circuit(corrected_circuit)
        post_correction_output = most_common_output(corrected_counts)
        print(f"For a bitflip error on qbit index {e_idx} in a circuit encoding the value {log_val}, the post-correction bitstring was {post_correction_output}")
        example_circ = circuit

example_circ.draw('text')

For a bitflip error on qbit index 0 in a circuit encoding the value 0, the post-correction bitstring was 000
For a bitflip error on qbit index 1 in a circuit encoding the value 0, the post-correction bitstring was 000
For a bitflip error on qbit index 2 in a circuit encoding the value 0, the post-correction bitstring was 000
For a bitflip error on qbit index 0 in a circuit encoding the value 1, the post-correction bitstring was 111
For a bitflip error on qbit index 1 in a circuit encoding the value 1, the post-correction bitstring was 111
For a bitflip error on qbit index 2 in a circuit encoding the value 1, the post-correction bitstring was 111


┌───┐ ░            ░       ░                      ░ 
q88_0: ┤ X ├─░───■────■───░───────░───■─────────■────────░─
       └───┘ ░ ┌─┴─┐  │   ░       ░   │         │        ░ 
q88_1: ──────░─┤ X ├──┼───░───────░───┼────■────┼────────░─
             ░ └───┘┌─┴─┐ ░ ┌───┐ ░   │    │    │        ░ 
q88_2: ──────░──────┤ X ├─░─┤ X ├─░───┼────┼────┼────■───░─
             ░      └───┘ ░ └───┘ ░ ┌─┴─┐┌─┴─┐  │    │   ░ 
q89_0: ─────────────────────────────┤ X ├┤ X ├──┼────┼───░─
                                    └───┘└───┘┌─┴─┐┌─┴─┐ ░ 
q89_1: ───────────────────────────────────────┤ X ├┤ X ├─░─
                                              └───┘└───┘ ░

We can see that this scheme works:
- No matter the logical value encoded (0 or 1).
- No matter which qbit index the error occurs on.

The quantum repetition code works.

## 5) A more elegant repetition code with the Toffoli gate
Up until now we have looked at a version that works. It's a bit clunky (we need to add 2 ancilla qbits, 2 cbits, apply corrections based on measurement results...) but it works. This approach is called **measurement-based QEC** or MBQEC and basically relies on measurement outcomes to determine the error correction action that must be taken.

However another version of the repetition code exists which doesn't need to add any ancilla qbits nor cbits and more importantly doesn't need to perform any measurements. It's leaner, faster (short-cuts the heavy measurement process) and in our opinion just more aesthetic. So why didn't we start with this one in the first place? Because it has one down side and it's that it's a less didactic way to do things. Since it's a more compact circuit, several steps are lumped into a single one and we don't get the nice step-by-step explanation of how to identify a syndrome and then apply corrections based on it as this is taken care of by the Toffoli gate itself. But the process is still there, syndrome identification and from there decision on the correction action is still present, but it's embedded in the Toffoli gate's truth table such that we don't get to do anything.

In [15]:
def toffoli_repetition_code_3(logical_qb_val:int=0, with_error:bool=False, qb_idx:int=-1) -> QuantumCircuit:
    # Encode the logical qubit information into three physical qubits
    qr1 = QuantumRegister(3)
    circuit = QuantumCircuit(qr1)

    if logical_qb_val == 1:
        circuit.x(0)
        circuit.barrier(qr1) #for visualisation only

    circuit = encode_1_into_3(circuit)
    circuit.barrier(qr1) #for visualisation only

    # If there is an error, introduce it here on deisred qbit
    if with_error:
        circuit.x(qb_idx)
        circuit.barrier(qr1) #for visualisation only

    # This step is necessary for the Toffoli-based RC and replaces MBQEC's parity check
    circuit.cnot(0,1)
    circuit.cnot(0,2)
    circuit.barrier(qr1) #for visualisation only
   
    # Add a Toffoli gate to correct potential errors
    circuit.ccx(2,1,0) # the Toffoli gate is also called CCNOT or CCX, they're all one and the same

    return circuit

c = toffoli_repetition_code_3(logical_qb_val=0)
c.draw('text')



░            ░ ┌───┐
q100_0: ──■────■───░───■────■───░─┤ X ├
        ┌─┴─┐  │   ░ ┌─┴─┐  │   ░ └─┬─┘
q100_1: ┤ X ├──┼───░─┤ X ├──┼───░───■──
        └───┘┌─┴─┐ ░ └───┘┌─┴─┐ ░   │  
q100_2: ─────┤ X ├─░──────┤ X ├─░───■──
             └───┘ ░      └───┘ ░

That's our new Toffoli-RC circuit. Slick, isn't it? See how compact it is without it's ancillas and classical bits?

In [16]:
def measure_toffoli_RC_circuit(circuit:QuantumCircuit, nb_shots:int=1024) -> Tuple[QuantumCircuit, dict]:
    c = deepcopy(circuit)
    
    cr = ClassicalRegister(1) # need one classical register to receive the measurement outcomes
    c.add_register(cr)
    
    c.measure(0,0)

    counts = simulate_measurements(c, nb_shots)

    return c, counts

And just like its MB counterpart, it works in all cases.

In [18]:
values_of_logical_qb = [0,1]
idx_of_error = [0,1,2]

example_circ = None
for log_val in values_of_logical_qb:
    for e_idx in idx_of_error:
        circuit = toffoli_repetition_code_3(logical_qb_val=log_val, with_error=True, qb_idx=e_idx)
        circuit, measurement_res = measure_toffoli_RC_circuit(circuit)
        res = most_common_output(measurement_res)
        print(f"For a bitflip error on qbit index {e_idx} on a circuit encoding {log_val} the corrected circuit yielded {res}")
        example_circ = circuit

example_circ.draw('text')


For a bitflip error on qbit index 0 on a circuit encoding 0 the corrected circuit yielded 0
For a bitflip error on qbit index 1 on a circuit encoding 0 the corrected circuit yielded 0
For a bitflip error on qbit index 2 on a circuit encoding 0 the corrected circuit yielded 0
For a bitflip error on qbit index 0 on a circuit encoding 1 the corrected circuit yielded 1
For a bitflip error on qbit index 1 on a circuit encoding 1 the corrected circuit yielded 1
For a bitflip error on qbit index 2 on a circuit encoding 1 the corrected circuit yielded 1


┌───┐ ░            ░       ░            ░ ┌───┐┌─┐
q167_0: ┤ X ├─░───■────■───░───────░───■────■───░─┤ X ├┤M├
        └───┘ ░ ┌─┴─┐  │   ░       ░ ┌─┴─┐  │   ░ └─┬─┘└╥┘
q167_1: ──────░─┤ X ├──┼───░───────░─┤ X ├──┼───░───■───╫─
              ░ └───┘┌─┴─┐ ░ ┌───┐ ░ └───┘┌─┴─┐ ░   │   ║ 
q167_2: ──────░──────┤ X ├─░─┤ X ├─░──────┤ X ├─░───■───╫─
              ░      └───┘ ░ └───┘ ░      └───┘ ░       ║ 
 c27: 1/════════════════════════════════════════════════╩═
                                                        0

## 6) Conclusion

In this chapter we have seen how to implement a simple QEC code called the repetition code to correct single bit-flip errors.
We have learnt:
- How to encode logical information into physical information using repetition.
- How to interpret and use error syndromes.
- How to implement both a measurement-based and a direct approach for the QRC.

In the next chapter, we'll see how to upgrade this RC to also correct for phase-flip errors.

Authors : Richard A. Wolf, ??? - License: CC-BY-SA-NC